In [2]:
import heapq
import time
import random
from dataclasses import dataclass, field
from typing import List, Tuple

# --- 1. ORDER OBJECT DESIGN ---
@dataclass(order=True)
class Order:
    # We use a tuple (priority, timestamp) for sorting in the heap.
    # We don't compare the other fields (qty, id) to keep it fast.
    sort_index: Tuple[float, float] = field(init=False, repr=False)
    
    id: int
    timestamp: float
    side: str # 'bid' or 'ask'
    price: float
    qty: int
    
    def __post_init__(self):
        # CRITICAL: For Bids, we negate price to simulate Max-Heap behavior
        # We add timestamp as a tie-breaker (FIFO)
        if self.side == 'bid':
            self.sort_index = (-self.price, self.timestamp)
        else:
            self.sort_index = (self.price, self.timestamp)

# --- 2. THE OLD WAY (LIST O(N)) ---
class ListOrderBook:
    def __init__(self):
        self.orders = []
        
    def add(self, order: Order):
        self.orders.append(order)
        # To maintain a "usable" book, we must sort it every time 
        # or scan it every time we want the best price.
        # Sorting is the standard "naive" implementation.
        self.orders.sort(key=lambda x: x.sort_index)
        
    def get_best(self):
        if not self.orders: return None
        return self.orders[0]

# --- 3. THE NEW WAY (HEAP O(Log N)) ---
class HeapOrderBook:
    def __init__(self):
        self.bids = [] # Max-Heap (via negation)
        self.asks = [] # Min-Heap
        
    def add(self, order: Order):
        # We push the whole object. The @dataclass(order=True) 
        # uses the .sort_index attribute automatically.
        if order.side == 'bid':
            heapq.heappush(self.bids, order)
        else:
            heapq.heappush(self.asks, order)
            
    def get_best_bid(self):
        if not self.bids: return None
        return self.bids[0] # O(1) Access

    def get_best_ask(self):
        if not self.asks: return None
        return self.asks[0] # O(1) Access

# --- 4. THE BENCHMARK ---
def run_benchmark(n_orders=10000):
    print(f"--- BENCHMARKING {n_orders} ORDERS ---")
    
    # Generate Synthetic Data
    orders_data = []
    for i in range(n_orders):
        side = 'bid' if random.random() > 0.5 else 'ask'
        price = random.uniform(90, 110)
        orders_data.append(Order(i, time.time(), side, price, 1))

    # --- RACE 1: LIST ---
    list_book = ListOrderBook()
    start_time = time.time()
    
    for o in orders_data:
        list_book.add(o) # Appends and Sorts
        _ = list_book.get_best() # Peeks
        
    list_duration = time.time() - start_time
    print(f"List (O(N log N)): {list_duration:.4f} seconds")

    # --- RACE 2: HEAP ---
    heap_book = HeapOrderBook()
    start_time = time.time()
    
    for o in orders_data:
        heap_book.add(o) # Heap Push
        if o.side == 'bid':
            _ = heap_book.get_best_bid()
        else:
            _ = heap_book.get_best_ask()
            
    heap_duration = time.time() - start_time
    print(f"Heap (O(log N)):   {heap_duration:.4f} seconds")
    
    # Conclusion
    speedup = list_duration / heap_duration
    print(f"\nResult: Heap is {speedup:.1f}x faster than List")

if __name__ == "__main__":
    run_benchmark(10000)

--- BENCHMARKING 10000 ORDERS ---
List (O(N log N)): 8.2032 seconds
Heap (O(log N)):   0.0155 seconds

Result: Heap is 529.7x faster than List
